결측치 존재 여부 확인

In [23]:
import pandas as pd

# 여러 인코딩 방식 시도
encodings = ['cp949', 'euc-kr', 'utf-8-sig']

for encoding in encodings:
    try:
        # CSV 파일 읽기
        df = pd.read_csv("C:/Users/Yeji/Desktop/서울시 전동킥보드 견인 현황.csv", encoding=encoding)
        
        # 결측치가 있는 행 찾기 (처리 전)
        missing_rows = df[df.isnull().any(axis=1)]
        print("=== 처리 전 결측치가 있는 행의 데이터 ===")
        print(missing_rows)
        break
        
    except UnicodeDecodeError:
        continue
    except Exception as e:
        print(f"오류 발생: {str(e)}")


=== 처리 전 결측치가 있는 행의 데이터 ===
         번호        신고일시  구정보                     주소   유형         조치일
79819  2816  2021-08-17  관악구  서울특별시 관악구 신림동 1467-10  NaN  1900-01-00


In [22]:
import pandas as pd

# 여러 인코딩 방식 시도
encodings = ['cp949', 'euc-kr', 'utf-8-sig']

for encoding in encodings:
    try:
        # CSV 파일 읽기
        df = pd.read_csv("C:/Users/Yeji/Desktop/서울시 전동킥보드 견인 현황.csv", encoding=encoding)

        # 열 이름 바꾸기
        df = df.rename(columns={'주소': '지번주소'})

        # 처리 전 행 수 출력
        print(f"필터링 전 총 행 수: {len(df)}")
        
        # 2021, 2022, 2023으로 시작하지 않는 행 찾기
        invalid_mask = ~df['조치일'].astype(str).str.startswith(('2021', '2022', '2023'))
        invalid_rows = df[invalid_mask]
        
        # 잘못된 데이터 출력
        print("\nInvalid rows indices:")
        print(invalid_rows.index.tolist())
        print("\nInvalid rows data:")
        print(invalid_rows)
        
        # 잘못된 데이터 삭제
        df = df[~invalid_mask]
        
        # 처리 후 행 수 출력
        print(f"\n필터링 후 총 행 수: {len(df)}")

        # 처리된 데이터프레임을 CSV 파일로 저장
        df.to_csv("C:/Users/Yeji/Desktop/서울시 전동킥보드 견인 현황_처리완료.csv", encoding=encoding, index=False)
        
        break
        
    except UnicodeDecodeError:
        continue
    except Exception as e:
        print(f"오류 발생: {str(e)}")

필터링 전 총 행 수: 129932

Invalid rows indices:
[79819]

Invalid rows data:
         번호        신고일시  구정보                   지번주소   유형         조치일
79819  2816  2021-08-17  관악구  서울특별시 관악구 신림동 1467-10  NaN  1900-01-00

필터링 후 총 행 수: 129931


지번 주소->도로명 주소 using KAKAO API

In [1]:
import requests
import json
import pandas as pd
import numpy as np

def jibun_doro(df):
    data = df.copy()
    data['도로명주소'] = 0
    data['행정동'] = 0
    data['경도'] = 0
    data['위도'] = 0
    
    for i in range(len(data)):
        if data.loc[i, '지번주소'] != '탈퇴':
            try:
                start = data.loc[i, '지번주소']
                url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(start)
                headers = {"Authorization": "KakaoAK " + "689a5fac1e585ade3c28dddc48390765"}
                place = requests.get(url, headers=headers).json()['documents']
                temp = place[0].get('address')   # 지번주소 정보 가져오기
                temp2 = place[0].get('road_address')   # 도로명주소 정보 가져오기 시도
                doro = temp2['address_name']
                dong = temp['region_3depth_h_name']
                long = temp['x']
                lat = temp['y']
                
                data.loc[i, ['도로명주소', '경도', '위도', '행정동']] = [doro, long, lat, dong]
                
            except:
                continue

    data.loc[data['도로명주소']==0, '도로명주소'] = np.nan
    return data

# CSV 파일 읽기 (인코딩은 파일에 맞게 설정)
df = pd.read_csv("C:/Users/Yeji/Desktop/서울시 전동킥보드 견인 현황_처리완료.csv", encoding='cp949')

# 함수 적용
result_df = jibun_doro(df)

# 결과 저장 (UTF-8 인코딩으로 저장)
result_df.to_csv("C:/Users/Yeji/Desktop/서울시 전동킥보드 견인 현황_도로명주소.csv", encoding='cp949', index=False)

C:\Users\Yeji\AppData\Local\Temp\ipykernel_23972\3607549802.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '서울 영등포구 국회대로70길 19' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, ['도로명주소', '경도', '위도', '행정동']] = [doro, long, lat, dong]
C:\Users\Yeji\AppData\Local\Temp\ipykernel_23972\3607549802.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '126.920819737553' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, ['도로명주소', '경도', '위도', '행정동']] = [doro, long, lat, dong]
C:\Users\Yeji\AppData\Local\Temp\ipykernel_23972\3607549802.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '37.5292331664388' has dtype incompatible with int64, please 

In [19]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv("C:/Users/Yeji/Desktop/서울시 전동킥보드 견인 현황_도로명주소.csv", encoding='cp949')

# 전체 데이터 수 계산
total_records = len(df)

# '도로명주소' 열의 결측치 수 계산
missing_road_address = df['도로명주소'].isna().sum()

# 결과 출력
print(f"전체 레코드 수: {total_records}")
print(f"도로명주소 결측치 수: {missing_road_address}")
print(f"도로명주소 결측치 비율: {missing_road_address/total_records*100:.2f}%")

전체 레코드 수: 129931
도로명주소 결측치 수: 87038
도로명주소 결측치 비율: 66.99%


In [1]:
import requests
import json
import pandas as pd
import numpy as np

def jibun_doro(df):
    data = df.copy()
    data['도로명주소'] = 0
    data['행정동'] = 0
    data['경도'] = 0
    data['위도'] = 0
    
    for i in range(len(data)):
        if data.loc[i, '지번주소'] != '탈퇴':
            try:
                start = data.loc[i, '지번주소']
                url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(start)
                headers = {"Authorization": "KakaoAK " + "689a5fac1e585ade3c28dddc48390765"}
                place = requests.get(url, headers=headers).json()['documents']
                temp = place[0].get('address')   # 지번주소 정보 가져오기
                
                # 도로명주소 예외처리
                try:
                    temp2 = place[0].get('road_address')
                    doro = temp2['address_name'] if temp2 else 0
                except:
                    doro = 0
            
                # 나머지 정보 추출
                dong = temp['region_3depth_h_name']
                long = temp['x']
                lat = temp['y']
            
                data.loc[i, ['도로명주소', '경도', '위도', '행정동']] = [doro, long, lat, dong]
            
            except:
                continue

    data.loc[data['도로명주소']==0, '도로명주소'] = np.nan
    return data

# CSV 파일 읽기 (인코딩은 파일에 맞게 설정)
df = pd.read_csv("C:/Users/Yeji/Desktop/서울시 전동킥보드 견인 현황_처리완료.csv", encoding='cp949')
# 함수 적용
result_df = jibun_doro(df)

# 결과 저장 (UTF-8 인코딩으로 저장)
result_df.to_csv("C:/Users/Yeji/Desktop/서울시 전동킥보드 견인 현황_도로명주소update.csv", encoding='cp949', index=False)

C:\Users\Yeji\AppData\Local\Temp\ipykernel_8240\2417394118.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '서울 영등포구 국회대로70길 19' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, ['도로명주소', '경도', '위도', '행정동']] = [doro, long, lat, dong]
C:\Users\Yeji\AppData\Local\Temp\ipykernel_8240\2417394118.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '126.920819737553' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, ['도로명주소', '경도', '위도', '행정동']] = [doro, long, lat, dong]
C:\Users\Yeji\AppData\Local\Temp\ipykernel_8240\2417394118.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '37.5292331664388' has dtype incompatible with int64, please exp

In [3]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv("C:/Users/Yeji/Desktop/서울시 전동킥보드 견인 현황_도로명주소update.csv", encoding='cp949')

# 전체 데이터 수 계산
total_records = len(df)

# '도로명주소' 열의 결측치 수 계산
missing_address = df['행정동'].isna().sum()

# 결과 출력
print(f"전체 레코드 수: {total_records}")
print(f"행정동 결측치 수: {missing_address}")
print(f"행정동 결측치 비율: {missing_address/total_records*100:.2f}%")

전체 레코드 수: 129931
행정동 결측치 수: 56
행정동 결측치 비율: 0.04%


C:\Users\Yeji\AppData\Local\Temp\ipykernel_8240\776503324.py:4: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/Yeji/Desktop/서울시 전동킥보드 견인 현황_도로명주소update.csv", encoding='cp949')


In [6]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv("C:/Users/Yeji/Desktop/서울시 전동킥보드 견인 현황_도로명주소update.csv", encoding='cp949')

# '행정동' 열이 결측치가 아닌 행만 필터링
filtered_df = df[df['행정동'].notna()]

# 필터링된 데이터를 새로운 CSV 파일로 저장
filtered_df.to_csv("C:/Users/Yeji/Desktop/서울시 전동킥보드 견인 현황_행정동_filtered.csv", index=False, encoding='cp949')

C:\Users\Yeji\AppData\Local\Temp\ipykernel_8240\3482417151.py:4: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/Yeji/Desktop/서울시 전동킥보드 견인 현황_도로명주소update.csv", encoding='cp949')


도로명 주소->도로명 코드